# Scraping Article URLs

1. Clean up what we actually scrape from the website: take out 
* related links e.g., RELATED
* advertisements e.g., FEATURED\
can take all paragraphs in the content column, then remove the columns that were in the featured / related elements from the paragraphs.

2. Clean up how we record the articles. There should be clear separation of paragraphs... probably one paragraph per cell.

In [2]:
import pandas as pd
import numpy as np
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import urllib.request 
from bs4 import NavigableString
from urllib.request import urlopen
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException
from selenium.webdriver.chrome.options import Options
import pprint


In [4]:
options = Options()
options.page_load_strategy = 'none'
driver = webdriver.Chrome(options=options)

# link = driver.find_element_by_class_name("load-more")


In [3]:
raw_headlines=[]
raw_links=[]
raw_dates=[]
domains=[
    'vaccines',
    'asia',
    'keyword/cell-gene-therapy',
    'keyword/drug-delivery',
    'facilities'
]
for domain in domains:
    driver.get(f"https://www.fiercepharma.com/{domain}")
    time.sleep(3)
    click=0
    for click in range(30):
        try:
            # time.sleep(2) #for page to load
            # link.click()
            time.sleep(2) #for page to load
            # driver.execute_script("window.scrollTo(500, document.body.scrollHeight);")
            # time.sleep() #for page to load
            # link = driver.find_element(By.XPATH,"//a[@title='Load more items']")
            link = driver.find_element(By.XPATH,"//div/span[@class='extended-load d-inline-block']")
            link.click()
            # time.sleep(2)
            click+=1
        except ElementClickInterceptedException:
            click+=0
        except StaleElementReferenceException:
            click+=0
        except:
            click+=1
    dates = driver.find_elements(By.CLASS_NAME, "date.d-inline-block")
    headlines1 = driver.find_elements(By.XPATH,"//h3[@class='element-title large']//a")
    headlines2 = driver.find_elements(By.XPATH,"//h3[@class='element-title xsmall']//a")
    headlines3 = driver.find_elements(By.XPATH,"//div[@class='element-title small']//a")
    for headline in headlines1:
        raw_links.append(headline.get_attribute("href"))
        raw_headlines.append(headline.text)
    for headline in headlines2:
        raw_links.append(headline.get_attribute("href"))
        raw_headlines.append(headline.text)
    for headline in headlines3:
        raw_links.append(headline.get_attribute("href"))
        raw_headlines.append(headline.text)
    for date in dates:
        raw_dates.append(date.text)
print(len(raw_headlines))
print(len(raw_dates))
print(len(raw_links))

1079
1067
1079


In [74]:
# test_links=test_links.split(',')
test_links[0]=test_links[0].replace('[','')
raw_links=[i.replace("'",'') for i in test_links]
raw_links

['https://www.fiercepharma.com/pharma/moderna-looks-scale-back-manufacturing-covid-booster-cope-falling-demand-reuters',
 ' https://www.fiercepharma.com/pharma/cdc-starts-wild-mild-campaign-reverse-falling-flu-vaccine-use-key-groups',
 ' https://www.fiercepharma.com/manufacturing/japan-awards-115m-arcturus-jv-boost-mrna-manufacturing',
 ' https://www.fiercepharma.com/marketing/biontech-and-x-warned-pr-body-trying-duck-covid-19-vaccine-debate',
 ' https://www.fiercepharma.com/manufacturing/astrazeneca-commits-mrna-signs-vaccine-manufacturing-deal-chinas-cansino',
 ' https://www.fiercepharma.com/pharma/echoing-pfizer-biontech-carefully-watching-cost-amid-declining-covid-vaccine-sales',
 ' https://www.fiercepharma.com/pharma/takeda-yanks-fda-application-dengue-vaccine-citing-data-collection-disagreement',
 ' https://www.fiercepharma.com/manufacturing/novavax-receive-350m-canada-canceled-covid-vaccine-orders',
 ' https://www.fiercepharma.com/marketing/fingerprint-spying-growing-vaccine-mar

In [3]:
article_list=list(pd.read_csv("working_list.csv")['article'])
problematic_list=list(pd.read_csv("problems.csv")['0'])
date_list=list(pd.read_csv("working_list.csv")['date'])
raw_links=list(pd.read_csv("ref.csv")['raw_links'])

In [10]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(raw_links[2])

(' '
 'https://www.fiercepharma.com/manufacturing/japan-awards-115m-arcturus-jv-boost-mrna-manufacturing')


In [4]:
problematic_list

[' https://www.fiercepharma.com/pharma/amid-china-launch-merck-cites-supply-limits-plan-to-scale-down-rotateq-shipments-africa',
 ' https://www.fiercepharma.com/pharma/as-promised-dupixent-and-vaccines-reboot-sanofi-back-to-growth',
 ' https://www.fiercepharma.com/vaccines/news-note-flu-vaccines-curevac-and-more',
 ' https://www.fiercepharma.com/vaccines/sanofi-s-dengvaxia-wins-fda-priority-review-despite-controversial-past',
 ' https://www.fiercepharma.com/vaccines/invectys-raises-eu15-million-series-a-to-fund-dna-cancer-vaccine-study-cll',
 ' https://www.fiercepharma.com/vaccines/merck-s-gardasil-has-a-monster-q3-unprecedented-global-demand-driving-growth',
 ' https://www.fiercepharma.com/vaccines/johnson-johnson-opens-eu72m-vaccine-plant-to-support-larger-clinical-trials-possible',
 ' https://www.fiercepharma.com/vaccines/news-note-altimmune-ebola-and-more',
 ' https://www.fiercepharma.com/vaccines/j-j-touts-latest-immune-response-data-for-mosaic-hiv-vaccine-program',
 ' https://www

In [18]:
len(article_list)+len(problematic_list)

978

In [34]:
#usual code to pull paragraphs

# article_list=pd.read_csv("working_list.csv")['article'][0]
# problematic_list=pd.read_csv("problems.csv")['problems'][0]
# date_list=pd.read_csv("working_list.csv")['date'][0]
# raw_links=pd.read_csv("working_list.csv")['raw_links'][0]
# article_list=[]
# problematic_list=[]
# date_list=[]
i=1
for url in problems[0]:
# for url in raw_links[len(article_list)+len(problematic_list):]:
        driver.get(url)
        try:
                time.sleep(4)
                # driver.execute_script("window.stop();")
                # paragraphs=driver.find_elements(By.TAG_NAME,"p")
                # paragraphs=driver.find_elements(By.XPATH,"//div[@class='col content']//p")
                url_paragraphs=[]
                url_paragraphs.extend([i.text for i in driver.find_elements(By.XPATH,"//div[@class='col content']//p")])
                # for paragraph in paragraphs:
                #         url_paragraphs.append(paragraph.text)
                article_list.append(url_paragraphs)
                date_list.append(driver.find_elements(By.XPATH,"//span[@class='date d-block d-md-inline-block']")[0].text)
                problematic_list.pop(url)
        except:
                pass
                # problematic_list.append(url)
        print(f"done with {i}")
        i+=1

done with 1
done with 2
done with 3
done with 4
done with 5
done with 6
done with 7
done with 8
done with 9
done with 10
done with 11
done with 12
done with 13
done with 14
done with 15
done with 16
done with 17
done with 18
done with 19
done with 20
done with 21
done with 22
done with 23
done with 24
done with 25
done with 26
done with 27
done with 28
done with 29
done with 30
done with 31
done with 32
done with 33
done with 34
done with 35
done with 36
done with 37
done with 38
done with 39
done with 40
done with 41
done with 42
done with 43
done with 44
done with 45
done with 46
done with 47
done with 48
done with 49
done with 50
done with 51
done with 52
done with 53
done with 54
done with 55
done with 56
done with 57
done with 58
done with 59
done with 60
done with 61
done with 62
done with 63
done with 64
done with 65
done with 66
done with 67
done with 68
done with 69
done with 70
done with 71
done with 72
done with 73
done with 74
done with 75
done with 76
done with 77
done wit

WebDriverException: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=117.0.5938.149)
Stacktrace:
0   chromedriver                        0x000000010043ad68 chromedriver + 4337000
1   chromedriver                        0x0000000100432de4 chromedriver + 4304356
2   chromedriver                        0x000000010005fa5c chromedriver + 293468
3   chromedriver                        0x0000000100047e68 chromedriver + 196200
4   chromedriver                        0x0000000100047c6c chromedriver + 195692
5   chromedriver                        0x00000001000460b8 chromedriver + 188600
6   chromedriver                        0x0000000100046bf4 chromedriver + 191476
7   chromedriver                        0x00000001000624b8 chromedriver + 304312
8   chromedriver                        0x00000001000dff60 chromedriver + 819040
9   chromedriver                        0x0000000100

In [20]:
# problematic list dedicated code

options = Options()
options.page_load_strategy = 'none'
driver = webdriver.Chrome(options=options)

# link = driver.find_element_by_class_name("load-more")

# article_list=pd.read_csv("working_list.csv")['article'][0]
# problematic_list=pd.read_csv("problems.csv")['problems'][0]
# date_list=pd.read_csv("working_list.csv")['date'][0]
# raw_links=pd.read_csv("working_list.csv")['raw_links'][0]
# article_list=[]
# problematic_list=[]
# date_list=[]
i=1
article_list=[]
test_problems=problematic_list[0:2]
for url in test_problems:
# for url in raw_links[len(article_list)+len(problematic_list):]:
        driver.get(url)
        time.sleep(15)
        driver.execute_script("window.stop();")
        # paragraphs=driver.find_elements(By.TAG_NAME,"p")
        # paragraphs=driver.find_elements(By.XPATH,"//div[@class='col content']//p")
        url_paragraphs=[]
        url_paragraphs.extend([i.text for i in driver.find_elements(By.XPATH,"//div[@class='col content']//p")])
        # for paragraph in paragraphs:
        #         url_paragraphs.append(paragraph.text)
        date_list.append(driver.find_elements(By.XPATH,"//span[@class='date d-block d-md-inline-block']")[0].text)
        article_list.append(url_paragraphs)
        problematic_list.pop(problematic_list.index(url))
        print(f"done with {i}")
        i+=1

done with 1
done with 2


In [24]:
article_list

["Like many CDMOs these days, Switzerland's Celonic aims to make a big splash in cell and gene therapy—and it's picked up space at an emerging Novartis life sciences hub to take that work to the next level. ",
 'Celonic sewed up a long-term lease for production and office space in the WST-222 building of Novartis’ hub-to-be in Stein, Switzerland, dubbed Life Science Park Rheintal. The company will use the 91,500-square-foot location to set up production capacity for cell and gene therapies, next-generation vaccines and “innovative biopharmaceuticals.”',
 '',
 'Celonic is adding up to 20 clean room suites, plus offices and laboratories for process development, method development and quality control for clients from early-stage work to market. It will also sign around 250 new employees to staff the site, which should be ready for production by the second quarter of 2022.',
 '',
 '',
 '',
 '',
 'RELATED: Cognate beefs up cell, gene therapy manufacturing with new plants in U.S., EU',
 'The

In [13]:
len(article_list)

1059

In [14]:
import pandas as pd
working_df=pd.DataFrame(data={'article':article_list,'date':date_list})
working_df.to_csv("working_list.csv")
# ref=pd.DataFrame(data={'headlines':raw_headlines,'raw_links':raw_links})
# ref.to_csv("ref.csv")
problems=pd.DataFrame(problematic_list)
problems.to_csv("problems.csv")

In [16]:
working_df

,article,date
0,['While the FDA and the Centers for Disease Co...,"Sep 13, 2023 10:25am"
1,['The Centers for Disease Control and Preventi...,"Sep 12, 2023 10:46am"
2,"[""Right after Daiichi Sankyo picked up Japan's...","Aug 14, 2023 11:58am"
3,['The self-regulatory body of the German commu...,"Aug 10, 2023 02:18am"
4,['Despite a Moderna partnership that turned so...,"Aug 9, 2023 11:28am"
...,...,...
1054,[Almost 90% of pharmaceutical companies see sm...,"May 11, 2021 06:40am"
1055,[Alexion Pharmaceuticals has wasted no time co...,"Oct 16, 2020 12:05pm"
1056,[The FDA just doled out the first nod under a ...,"Oct 1, 2018 12:20pm"
1057,"[Ten23 health, a CDMO that launched just six m...","Mar 28, 2022 09:30am"


In [25]:
working_df.rename(columns={'date':'datetime'},inplace=True)
working_df['date']=pd.to_datetime(working_df['datetime'], format='mixed').dt.date

,article,datetime,date
0,['While the FDA and the Centers for Disease Co...,"Sep 13, 2023 10:25am",2023-09-13
1,['The Centers for Disease Control and Preventi...,"Sep 12, 2023 10:46am",2023-09-12
2,"[""Right after Daiichi Sankyo picked up Japan's...","Aug 14, 2023 11:58am",2023-08-14
3,['The self-regulatory body of the German commu...,"Aug 10, 2023 02:18am",2023-08-10
4,['Despite a Moderna partnership that turned so...,"Aug 9, 2023 11:28am",2023-08-09
...,...,...,...
1054,[Almost 90% of pharmaceutical companies see sm...,"May 11, 2021 06:40am",2021-05-11
1055,[Alexion Pharmaceuticals has wasted no time co...,"Oct 16, 2020 12:05pm",2020-10-16
1056,[The FDA just doled out the first nod under a ...,"Oct 1, 2018 12:20pm",2018-10-01
1057,"[Ten23 health, a CDMO that launched just six m...","Mar 28, 2022 09:30am",2022-03-28


In [27]:
working_df=working_df.sort_values(by="date")
working_df.to_csv("all_articles.csv")

In [27]:
working_df=pd.DataFrame(data={'article':article_list})
working_df.to_csv("test.csv")
working_df['article'][0]

['For developers of cell and gene therapies, the path to market often seems to be marred with potholes. The FDA rejected BioMarin Pharmaceutical’s gene therapy to treat hemophilia A in 2020, citing the need for more durability data and sending the company’s shares down more than 20% in a day. In November of this year, the FDA delayed a much-anticipated decision on multiple myeloma CAR-T candidate cilta-cel from Legend Biotech and Johnson & Johnson, causing Legend’s stock to fall nearly 8%.',
 'Now, those companies are preparing for the FDA to finally hand down its rulings—and they’re making those preparations optimistically. Biomarin expects to submit new data on its gene therapy, Roctavian, to the FDA in 2022, setting up a potential approval by the end of the year. And, at this year’s American Society of Hematology (ASH) meeting, Legend and J&J rolled out such positive survival data from their cilta-cel trial that some analysts suggested—if it’s approved in February as expected—it cou

In [33]:
result=pd.read_csv("test.csv")
pp.pprint(result['article'][0])

("['For developers of cell and gene therapies, the path to market often seems "
 'to be marred with potholes. The FDA rejected BioMarin Pharmaceutical’s gene '
 'therapy to treat hemophilia A in 2020, citing the need for more durability '
 'data and sending the company’s shares down more than 20% in a day. In '
 'November of this year, the FDA delayed a much-anticipated decision on '
 'multiple myeloma CAR-T candidate cilta-cel from Legend Biotech and Johnson & '
 "Johnson, causing Legend’s stock to fall nearly 8%.', 'Now, those companies "
 'are preparing for the FDA to finally hand down its rulings—and they’re '
 'making those preparations optimistically. Biomarin expects to submit new '
 'data on its gene therapy, Roctavian, to the FDA in 2022, setting up a '
 'potential approval by the end of the year. And, at this year’s American '
 'Society of Hematology (ASH) meeting, Legend and J&J rolled out such positive '
 'survival data from their cilta-cel trial that some analysts suggeste